In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import traceback
import numpy as np
import datatable as dt
import gresearch_crypto
import pandas as pd
import datatable as dt
dataset = dt.fread('../input/g-research-crypto-forecasting/train.csv').to_pandas()
dataset.head()

In [ ]:
assetDetails = dt.fread('/kaggle/input/g-research-crypto-forecasting/asset_details.csv').to_pandas().sort_values('Asset_ID')
assetDetails

In [ ]:
def upperShadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lowerShadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def getFeatures(df):
    modified_df = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 
                      'VWAP']].copy()
    modified_df['Upper Shadow'] = upperShadow(modified_df)
    modified_df['Lower Shadow'] = lowerShadow(modified_df)
    return modified_df

In [ ]:
import xgboost as xgb

def get_Xy_and_model_for_asset(dfTrain, assetId):
    df = dfTrain[dfTrain['Asset_ID'] == assetId]
    df_proc = getFeatures(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how='any')
    
    X = df_proc.drop('y', axis=1)
    y = df_proc['y']
    
    model = xgb.XGBRegressor(
        n_estimators=500,
        max_depth=11,
        learning_rate=0.06,
        subsample=0.9,
        colsample_bytree=0.75,
        missing=-999,
        random_state=2022
        #tree_method='gpu_hist'
    )
    
    model.fit(X,y)
    return X, y, model


In [ ]:
Xs ={}
ys ={}
models ={}

for assetId, assetName in zip(assetDetails['Asset_ID'], 
                              assetDetails['Asset_Name']):
    print(f"Training model for {assetName:<16} (ID={assetId:<2})")
    try:
        X, y, model = get_Xy_and_model_for_asset(dataset, assetId)
        Xs[assetId], ys[assetId], models[assetId] = X, y, model
    except:
        Xs[assetId], ys[assetId], models[assetId] = None, None, None
        
        

In [ ]:
x = getFeatures(dataset.iloc[1])
y_pred = models[0].predict(pd.DataFrame([x]))
y_pred[0]

In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        if models[row['Asset_ID']] is not None:
            try:
                model = models[row['Asset_ID']]
                x_test = getFeatures(row)
                y_red = model.predict(pd.DataFrame([x_test]))[0]
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
            except:
                df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
                traceback.print_exc()
        else: 
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
        
    env.predict(df_pred)